检查给codebook引入Padding之后是否会影响模型性能

In [3]:
import random
import os
import transformers
import sys
sys.path.append("/workspace/zecheng/modelzipper/projects/custom_llama")
from dataclasses import dataclass, field
from transformers import Trainer
from modelzipper.tutils import *
from data.vqllama_dataset import VQDataCollator, VQLLaMAData
from models.vqvae import VQVAE
from data.svg_data import *
import pytorch_lightning as pl

VQVAE_CONFIG_PATH = "/workspace/zecheng/modelzipper/projects/custom_llama/configs/deepspeed/vqvae_config.yaml"
DATA_PATH = "/zecheng2/svg/icon-shop/test_data_snaps/test_mesh_data_svg_convert_p.pkl"

content = auto_read_data(DATA_PATH)
dataset = BasicDataset(dataset=content)
vqvae_config = load_yaml_config(VQVAE_CONFIG_PATH)

block_kwargs = dict(
        width=vqvae_config.vqvae_conv_block.width, 
        depth=vqvae_config.vqvae_conv_block.depth, 
        m_conv=vqvae_config.vqvae_conv_block.m_conv,
        dilation_growth_rate=vqvae_config.vqvae_conv_block.dilation_growth_rate,
        dilation_cycle=vqvae_config.vqvae_conv_block.dilation_cycle,
        reverse_decoder_dilation=vqvae_config.vqvae_conv_block.vqvae_reverse_decoder_dilation
    )

class PluginVQVAE(nn.Module):
    def __init__(self, model):
        super().__init__()
        self.model = model

vqvae = VQVAE(vqvae_config, multipliers=None, **block_kwargs)
plugin_vqvae = PluginVQVAE(vqvae)
checkpoint = torch.load(vqvae_config.ckpt_path)  # load vqvae ckpt
plugin_vqvae.load_state_dict(checkpoint['state_dict'])
plugin_vqvae.eval()
plugin_vqvae.cpu()

begin to read data from /zecheng2/svg/icon-shop/test_data_snaps/test_mesh_data_svg_convert_p.pkl ...
begin to sanity check the dataset and conduct pre_process, num of samples: 2000, it will take some time...
load config files from /workspace/zecheng/modelzipper/projects/custom_llama/configs/deepspeed/vqvae_config.yaml
config loaded successfully!
config: namespace(ckpt_path='/zecheng2/vqllama/vqllama_quantizer/version_8/checkpoints/last.ckpt', pad_token_id=70, vqvae=namespace(levels=2, downs_t=[1, 1], strides_t=[2, 2], emb_width=4096, l_bins=8192, l_mu=0.99, spectral=0.0, multispectral=1.0, hvqvae_multipliers=[2, 1, 1], loss_fn='l2', dilation_growth_rate=1, use_nonrelative_specloss=True, use_bottleneck=True, commit=1.0, recon=1.0, linf_k=2048, use_modified_block=False), vqvae_conv_block=namespace(depth=4, width=512, m_conv=1.0, dilation_growth_rate=1, dilation_cycle=None, vqvae_reverse_decoder_dilation=True), dataset=namespace(max_path_nums=512, min_path_nums=4, pad_token_id=0, train_ba

<All keys matched successfully>

In [32]:
sample = dataset[0]['svg_path']
max_seq_len = 512
padded_sample = torch.concatenate([sample, torch.zeros(max_seq_len - sample.shape[0], 9)])
padding_mask = padded_sample == 0
svg_token_ids, _ = plugin_vqvae.model.encode(padded_sample.unsqueeze(0), start_level=0, end_level=1)
svg_token_ids = svg_token_ids[0]

In [35]:
padding_mask = ~(padded_sample == 0).all(dim=1, keepdim=True).squeeze()

In [36]:
padding_mask

tensor([ True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True, 